In [7]:
import pandas as pd
import numpy as np


In [36]:
# inputs
num_records = 1_000 #10_000_000


# initalize df
order_df = pd.DataFrame({"CUSTOMER_ID": np.random.randint(1, 100, size=(num_records)),
                         "ORDER_ID": np.random.randint(1, 7, size=(num_records)), 
                         "PROD_ID": np.random.randint(1, 1000, size=(num_records)), 
                         "ORDER_QTY": np.random.randint(1, 20, size=(num_records))
                        })
print(f"Number of Rows: {order_df.shape[0]}")
order_df.head()

Number of Rows: 1000


,CUSTOMER_ID,ORDER_ID,PROD_ID,ORDER_QTY
0,99,4,655,13
1,47,5,158,13
2,71,3,89,19
3,74,2,587,16
4,61,6,472,2


In [130]:
# Group by Customer Id 
grpdOrder_df = order_df.groupby(['CUSTOMER_ID'])

# Loop through cust ids
for k, custId in grpdOrder_df:
    # 1. Get unique prod ids to and use them to pad 
    unique_Order_Ids = custId['PROD_ID'].unique()
    print(f"unique_Order_Ids {unique_Order_Ids}")
    

unique_Order_Ids [521 543 608 719 449 784 137 589]
unique_Order_Ids [ 76 412 958]
unique_Order_Ids [664 394 381 472 312 694 829 285 665 555 852 527  41]
unique_Order_Ids [308 191 450 662 915 893 306 159 280 811 510 609 354 211]
unique_Order_Ids [236 466 640 727 203 974 921 769 348 380 463 814 454]
unique_Order_Ids [950 268 657 728 277 178 161 633 396 561 704 401 613]
unique_Order_Ids [741 544 801  79 380 942  10 963  35]
unique_Order_Ids [196 320 461 303 885 152 636 939]
unique_Order_Ids [439 449 938  92 843 275 595 276 844 861 441 763]
unique_Order_Ids [922 830 841 491 294 785 291 864 139]
unique_Order_Ids [798 388 313 112 570 139 197]
unique_Order_Ids [581 907 844 307 852 408 488]
unique_Order_Ids [470 605 577 268 389 465]
unique_Order_Ids [873 827 455 555 617 619  49 470 215 750 724 165]
unique_Order_Ids [225 171 945 335 395 857 989 626  29 147]
unique_Order_Ids [451 986 163  33 877 197]
unique_Order_Ids [812 228 926 901 214]
unique_Order_Ids [773 909 104 309 699 552 689 366 795 149

In [41]:
grpdOrder_df = order_df.groupby(['CUSTOMER_ID'])

In [106]:
grpd_lst = [g[1] for g in list(grpdOrder_df)]
df = grpd_lst[0]
df

,CUSTOMER_ID,ORDER_ID,PROD_ID,ORDER_QTY
67,1,5,521,6
335,1,1,543,5
346,1,1,608,4
492,1,4,719,17
515,1,2,449,19
578,1,2,784,7
700,1,3,137,9
935,1,4,589,5


In [107]:
# 1. Get unique prod ids to and use them to pad 
unique_Order_Ids = df['PROD_ID'].unique()
unique_Order_Ids

array([521, 543, 608, 719, 449, 784, 137, 589])

In [120]:
df_fill  = pd.DataFrame(unique_Order_Ids, columns=['PROD_ID'])

In [123]:
df = df.set_index("PROD_ID")

In [124]:
df.join(df_fill, on=["PROD_ID"] ,how='outer' )

,CUSTOMER_ID,ORDER_QTY,PROD_ID
PROD_ID,,,
521,1.0,6.0,521
543,1.0,5.0,543
608,1.0,4.0,608
719,1.0,17.0,719
449,1.0,19.0,449
784,1.0,7.0,784
137,1.0,9.0,137
589,1.0,5.0,589
589,NaN,NaN,521


In [97]:
new_index = pd.Index(unique_Order_Ids, name="PROD_ID")
new_index

Int64Index([521, 543, 608, 719, 449, 784, 137, 589], dtype='int64', name='PROD_ID')

In [115]:
df.set_index("PROD_ID").reindex(new_index)

,CUSTOMER_ID,ORDER_ID,ORDER_QTY
PROD_ID,,,
521,1,5,6
543,1,1,5
608,1,1,4
719,1,4,17
449,1,2,19
784,1,2,7
137,1,3,9
589,1,4,5


In [68]:
g = df.groupby(['ORDER_ID'])
for k, grp in g:
    print(grp)
    
    # Get the prod ids that were not ordered (order id)
    prodId_pad_lst = list(set(unique_Order_Ids)-set(grp["PROD_ID"]))
    print(prodId_pad_lst)
    
    if len(prodId_pad_lst) > 0:
        for prod in prodId_pad_lst:
            tmp_df = pd.
        
    

     CUSTOMER_ID  ORDER_ID  PROD_ID  ORDER_QTY
335            1         1      543          5
346            1         1      608          4
[449, 137, 521, 589, 719, 784]
     CUSTOMER_ID  ORDER_ID  PROD_ID  ORDER_QTY
515            1         2      449         19
578            1         2      784          7
[608, 521, 137, 589, 719, 543]
     CUSTOMER_ID  ORDER_ID  PROD_ID  ORDER_QTY
700            1         3      137          9
[608, 449, 521, 589, 719, 784, 543]
     CUSTOMER_ID  ORDER_ID  PROD_ID  ORDER_QTY
492            1         4      719         17
935            1         4      589          5
[608, 449, 137, 521, 784, 543]
    CUSTOMER_ID  ORDER_ID  PROD_ID  ORDER_QTY
67            1         5      521          6
[608, 449, 137, 589, 719, 784, 543]


In [70]:
df1 = pd.DataFrame({"A":[0,0.5,1.0,3.5,4.0,4.5], "B":[1,4,6,2,4,3], "C":[3,2,1,0,5,3]})
df1

,A,B,C
0,0.0,1,3
1,0.5,4,2
2,1.0,6,1
3,3.5,2,0
4,4.0,4,5
5,4.5,3,3


In [90]:
pd.arange(0,5,0.5)

AttributeError: module 'pandas' has no attribute 'arange'

In [34]:



order_df.groupby(['CUSTOMER_ID'])['ORDER_ID'].unique()

CUSTOMER_ID
1     [3, 5, 6, 1, 2, 4]
2     [2, 5, 6, 4, 1, 3]
3     [5, 4, 1, 3, 2, 6]
4     [2, 6, 5, 1, 4, 3]
5     [4, 3, 2, 5, 6, 1]
6     [5, 4, 1, 3, 6, 2]
7     [4, 5, 1, 2, 6, 3]
8     [6, 5, 1, 2, 4, 3]
9     [5, 4, 3, 6, 2, 1]
10    [3, 5, 6, 1, 2, 4]
11    [4, 2, 5, 1, 6, 3]
12    [6, 4, 1, 3, 2, 5]
13    [3, 6, 5, 1, 2, 4]
14    [3, 4, 6, 2, 1, 5]
15    [5, 2, 4, 3, 6, 1]
16    [4, 5, 1, 6, 3, 2]
17    [5, 1, 3, 4, 6, 2]
18    [5, 6, 4, 2, 1, 3]
19    [5, 2, 6, 3, 1, 4]
20    [1, 6, 4, 3, 2, 5]
21    [3, 4, 1, 6, 2, 5]
22    [1, 4, 2, 3, 6, 5]
23    [4, 5, 1, 2, 6, 3]
24    [2, 3, 5, 6, 4, 1]
25    [3, 5, 1, 4, 6, 2]
26    [1, 3, 2, 4, 5, 6]
27    [6, 5, 1, 2, 4, 3]
28    [3, 6, 2, 4, 1, 5]
29    [2, 4, 1, 3, 5, 6]
30    [3, 2, 6, 1, 5, 4]
             ...        
70    [6, 5, 4, 1, 2, 3]
71    [4, 1, 6, 2, 5, 3]
72    [2, 6, 4, 5, 3, 1]
73    [1, 5, 2, 6, 3, 4]
74    [5, 6, 4, 1, 3, 2]
75    [4, 5, 2, 3, 6, 1]
76    [1, 4, 5, 2, 3, 6]
77    [3, 2, 1, 4, 5, 6]
78    [3, 6, 

In [30]:
# Pad order table for predict order pipeline

# # 1. Group by custid
order_grpd = order_df.groupby(['CUSTOMER_ID'])

# # 2. Loop through groups and find unique prod id's and 
for k, g in order_grpd:
    print(g)

     CUSTOMER_ID  ORDER_ID  PROD_ID  ORDER_QTY
3              1         1      232         17
72             1         2       82         11
179            1         1      631         16
229            1         6      503         19
260            1         3      741          8
364            1         5      116          9
446            1         6      495          1
667            1         5      888         17
822            1         6      146         10
879            1         2       84         19
984            1         4      533          4
     CUSTOMER_ID  ORDER_ID  PROD_ID  ORDER_QTY
248            2         6       17          4
328            2         3      261         19
596            2         2      681          9
790            2         6       14         15
855            2         4      813          8
982            2         6      392         18
     CUSTOMER_ID  ORDER_ID  PROD_ID  ORDER_QTY
57             3         3      896          4
193          